In [1]:
from pysy import *

print(dir())

['Canvas', 'EEarth', 'Ecolbox', 'Emagebox', 'Geometry', 'In', 'Out', 'Utils', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'exit', 'fluxlib', 'get_ipython', 'quit', 'scigee', 'scigeo', 'utils', 'viz_params']


In [2]:
def get_evi_value(earth, band, scale = 10, default_value = -9999.):
    for i in range(earth.__length__):
        image = earth.get_image_by_index(idx = i)
        imgbox = Emagebox(image, scale = scale)
        val = imgbox.get_value(band, point)
        vals = imgbox.get_values(band, roi)["values"]
        if np.any(vals != default_value):
            mean_val = np.nanmean(np.where(vals == default_value, np.nan, vals))
        else:
            mean_val = None
        if val:
            val = np.round(val, 4)
        if mean_val:
            mean_val = np.round(mean_val, 4)
        print(val, mean_val, imgbox.get_date())

In [ ]:
from pysy import *
import numpy as np

landsat_7_evi = "LANDSAT/LE07/C01/T1_8DAY_EVI"
landsat_8_evi = "LANDSAT/LC08/C01/T1_8DAY_EVI"
sentinel_2_TOA = "COPERNICUS/S2"

date_range = ["2018-6-1", "2018-10-11"]
# Ameriflux: US-ARM
lat = 36.6058
lon = -97.4888

geometry = Geometry(lon = lon, lat = lat)
point = geometry(0)
roi = Geometry.get_rect_buffer(point, buffer_size = 0.001)


s2toa = Ecolbox(sentinel_2_TOA, date_range = date_range, roi = roi)

# cloud mask
s2toa_imgs = s2toa.collection.map(
    lambda image: Utils.sentinel_2_cloud_mask(image)
)

# calculate CIRE
s2toa_imgs = s2toa_imgs.map(
    lambda image: Utils.calc_vi("cire", image, re2 = "B6", re3 = "B7")
)

# calculate PRI
s2toa_imgs = s2toa_imgs.map(
    lambda image: Utils.calc_vi("ndi", image, b1 = "B2", b2 = "B3", label = "PRI")
)

print(f"sentinel2 CIRE {'-'*20}")
# update s2toa's attr collection
s2toa(s2toa_imgs)
get_evi_value(s2toa, "CIRE")
print(f"sentinel2 PRI {'-'*20}")
get_evi_value(s2toa, "PRI")
    
# # Example of calculating evi from a Lansat8 image
# image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
# evi = VI("evi", image, nir = "B5", red = "B4", blue = "B3").evi

print(f"landsat7 {'-'*20}")   
ls7evis = Ecolbox(landsat_7_evi, date_range = date_range, roi = roi)
get_evi_value(ls7evis, "EVI")
    
print(f"landsat8 {'-'*20}")
ls8evis = Ecolbox(landsat_8_evi, date_range = date_range, roi = roi)
get_evi_value(ls8evis, "EVI")
    
    
# ls7evis = Ecolbox(landsat_7_evi, date_range = date_range, roi = roi)
# image = ls7evis.get_image_by_index(idx = 0)
# imgbox = Emagebox(image, scale = 10)
# print(imgbox.get_value("EVI", point))
# vals = imgbox.get_values("EVI", roi)["values"]
# print(np.nanmean(np.where(vals==-9999., np.nan, vals))) 
# canvas = Canvas()
# canvas.draw([image, roi, point], [canvas.viz(0, 1, canvas.cmap["veg"]), {}, {}], layer_names = ["evi", "poly", "site"], location = [lat, lon], zoom_start = 7)
# display(canvas.m)

print("ok")

sentinel2 --------------------
None None 2018-6-4
None None 2018-6-9
0.2177 0.1861 2018-6-14
0.1418 0.197 2018-6-19
None None 2018-6-24
0.237 0.2245 2018-6-29
0.2583 0.2335 2018-7-4
0.165 0.1942 2018-7-9
None None 2018-7-14
0.1843 0.1698 2018-7-19
None None 2018-7-24
0.1868 0.1934 2018-7-29
0.191 0.1656 2018-8-3
None None 2018-8-8
None None 2018-8-13
0.2056 0.2013 2018-8-18
None None 2018-8-23
None None 2018-8-28
0.1915 0.1872 2018-9-2
None None 2018-9-7
None None 2018-9-12
0.2606 0.2509 2018-9-17
0.1957 0.1703 2018-9-22
0.1962 0.1735 2018-9-27
0.2115 0.1791 2018-10-2
None None 2018-10-7
None None 2018-6-4
None None 2018-6-9
0.0443 0.0047 2018-6-14
0.0558 0.026 2018-6-19
None None 2018-6-24
-0.0093 0.0029 2018-6-29
-0.0171 0.0109 2018-7-4
0.128 0.147 2018-7-9
None None 2018-7-14
0.0056 -0.0009 2018-7-19
None None 2018-7-24
0.0203 0.0285 2018-7-29
0.0074 0.0056 2018-8-3
None None 2018-8-8
None None 2018-8-13
0.0431 0.0146 2018-8-18
None None 2018-8-23
None None 2018-8-28
0.0579 0.0099 2

In [35]:
class A(object):
    def __new__(cls):
        print("__new__ is called")
        return super(A, cls).__new__(cls)

    def __init__(self):
        super(object, self).__init__()
        self.attr1 = "dog"
        print("__init__ is called")
        
    def __getattribute__(self, attr):
        print ("A.__getattribute__ called")
        return super(A, self).__getattribute__(attr)
    
    def __getattr__(self, attr):
        attr = "cat"
        print ("A.__getattr__ called")
        return attr

a = A()
# print(a.attr1)
# print(a.attr2)
print(a.attr3)
print(dir(a))

__new__ is called
__init__ is called
A.__getattribute__ called
A.__getattr__ called
cat
A.__getattribute__ called
A.__getattribute__ called
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attr1']
